## SQL Notebook for Current Year Database

In [5]:
## Dependencies
import os
import sys
import time

import numpy as np
import pandas as pd
import sqlite3

# Database Path
db_path = os.path.join('..', 'data', 'db', 'Jan_29_Current_YTD_Game_Stats.db')

# Connect to the database
conn = sqlite3.connect(db_path)



In [7]:
# Get the schema of the scoring_summary table
schema_query = "PRAGMA table_info(scoring_summary);"
scoring_summary_schema = conn.execute(schema_query).fetchall()

# Display the schema of the scoring_summary table
scoring_summary_schema


[(0, 'Period', 'TEXT', 0, None, 0),
 (1, 'Team', 'TEXT', 0, None, 0),
 (2, 'PP', 'TEXT', 0, None, 0),
 (3, 'Player', 'TEXT', 0, None, 0),
 (4, 'Player_Goals', 'BIGINT', 0, None, 0),
 (5, 'Assist1', 'TEXT', 0, None, 0),
 (6, 'Assist2', 'TEXT', 0, None, 0),
 (7, 'Time', 'TEXT', 0, None, 0),
 (8, 'Game_ID', 'TEXT', 0, None, 0)]

In [11]:
# Reconnect to the database to perform the analysis
conn = sqlite3.connect(db_path)

# SQL to create a temporary view for empty net goals scored (EN goals for)
create_en_goals_for_view = """
CREATE TEMP VIEW IF NOT EXISTS en_goals_for AS
SELECT Team, COUNT(*) AS EN_Goals_For
FROM scoring_summary
WHERE PP = 'EN'
GROUP BY Team;
"""

# SQL to create a temporary view for empty net goals allowed (EN goals against)
create_en_goals_against_view = """
CREATE TEMP VIEW IF NOT EXISTS en_goals_against AS
SELECT Team, COUNT(*) AS EN_Goals_Against
FROM scoring_summary
WHERE PP = 'EN'
GROUP BY Team;
"""

# Execute the SQL commands to create the views
conn.execute(create_en_goals_for_view)
conn.execute(create_en_goals_against_view)

# Now, combine these views to create a summary table for empty net goals for and against each team
en_goals_summary_query = """
SELECT 
    f.Team AS Team,
    f.EN_Goals_For,
    a.EN_Goals_Against
FROM 
    en_goals_for f
JOIN 
    en_goals_against a ON f.Team = a.Team;
"""

# Execute the query to get the empty net goals summary
en_goals_summary = conn.execute(en_goals_summary_query).fetchall()

# Since there might be a mistake in how we calculate "against" (it's actually the same calculation as "for" at the moment),
# let's just display the "for" results for now and then correct our approach.
en_goals_for_query = """
SELECT Team, COUNT(*) AS EN_Goals_For
FROM scoring_summary
WHERE PP = 'EN'
GROUP BY Team;
"""

en_goals_for = conn.execute(en_goals_for_query).fetchall()

# Close the database connection
conn.close()

# Sort the results by the number of empty net goals for
en_goals_for.sort(key=lambda x: x[1], reverse=True)

# Display the results for empty net goals "for", to verify our approach before correcting the "against" part
en_goals_for


[('WMU', 11),
 ('HC', 9),
 ('ASU', 6),
 ('Clarkson', 6),
 ('MSU', 6),
 ('BC', 5),
 ('Maine', 5),
 ('NDame', 5),
 ('RIT', 5),
 ('SHU', 5),
 ('UND', 5),
 ('Wisconsin', 5),
 ('AIC', 4),
 ('Air Force', 4),
 ('Bentley', 4),
 ('CC', 4),
 ('Cornell', 4),
 ('LSSU', 4),
 ('Minnesota', 4),
 ('NMU', 4),
 ('Northeastern', 4),
 ('PSU', 4),
 ('RMU', 4),
 ('UConn', 4),
 ('UMD', 4),
 ('BSU', 3),
 ('Canisius', 3),
 ('Denver', 3),
 ('MnSU', 3),
 ('OSU', 3),
 ('QU', 3),
 ('UMass', 3),
 ('UVM', 3),
 ('Alaska', 2),
 ('Army', 2),
 ('BGSU', 2),
 ('BU', 2),
 ('Brown', 2),
 ('FSU', 2),
 ('MTU', 2),
 ('Merrimack', 2),
 ('Miami', 2),
 ('Niagara', 2),
 ('PC', 2),
 ('RPI', 2),
 ('SLU', 2),
 ('UNH', 2),
 ('UST', 2),
 ('Union', 2),
 ('Augustana', 1),
 ('Colgate', 1),
 ('LIU', 1),
 ('Lindenwood', 1),
 ('Michigan', 1),
 ('Princeton', 1),
 ('SCSU', 1),
 ('UML', 1),
 ('Yale', 1)]